# Data Collection Pipeline
This Script Loads all data sources needed for further data exploration.
### Datasets
    Kaggle UEFA: https://www.kaggle.com/datasets/azminetoushikwasi/ucl-202122-uefa-champions-league
    TransferMarkt Website: https://www.transfermarkt.com/schnellsuche/ergebnis/schnellsuche
    Wikipedia National Crime Stats: https://en.wikipedia.org/wiki/List_of_countries_by_intentional_homicide_rate

### IMPORTANT
**Make sure Docker is up and running!**

### Steps
0. Prepare local backup strateyg for mongo with a docker container due to problems with the hosted mongo instance
1. Download UEFA Dataset and store information in MongoDB
2. Iterate over UEFA Dataset player names and Scrape additional information from Transfermarkt.com
3. Cleanup combined dataset
4. Scrape Wikipedia to get crime information per country and store it in MongoDB

In [2]:
!docker pull mongo

Using default tag: latest
latest: Pulling from library/mongo
Digest: sha256:9c8a0a019671ed7d402768d4df6dddcc898828e21e9f7b90a34b55fe8ca676ac
Status: Image is up to date for mongo:latest
docker.io/library/mongo:latest


## If container does not exist yet

In [9]:
!docker run -d -p 27017:27017 --name=mongo mongo:latest

docker: Error response from daemon: Conflict. The container name "/mongo" is already in use by container "8a84526dbb09f9c5f99228f621514d28c058bc0cd924c0115efa49233f208af3". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.


## If container already exists

In [11]:
!docker start mongo

mongo


In [13]:
!docker ps

CONTAINER ID   IMAGE          COMMAND                  CREATED          STATUS          PORTS                      NAMES
8a84526dbb09   mongo:latest   "docker-entrypoint.s…"   53 minutes ago   Up 50 minutes   0.0.0.0:27017->27017/tcp   mongo


In [19]:
import pandas as pd
import zipfile

import requests

try:
    from pymongo import MongoClient
    from pymongo.errors import ServerSelectionTimeoutError
except ImportError:
    !pip install pymongo[srv]
    from pymongo import MongoClient
    from pymongo.errors import ServerSelectionTimeoutError

from conifg import MONGO_HOST_REMOTE, MONGO_DB_REMOTE, MONGO_HOST_LOCAL, MONGO_DB_LOCAL

UEFA_ZIP = "kaggle_players_zip.zip"
UEFA_UNZIPPED = "kaggle_files"
UEFA_FILES = ["key_stats.csv", "disciplinary.csv", "distributon.csv", "defending.csv"]
UEFA_RAW_DATA = "raw_players"

conn_str = MONGO_HOST_REMOTE
mongoDB = MONGO_DB_REMOTE

class MongoContext:
    """mongodb client context manager"""

    def __enter__(self):
        global conn_str, mongoDB
        try:
            self.client = MongoClient(conn_str)
            self.client.server_info()
            #print("Connection successful to remote mongo host")
        except ServerSelectionTimeoutError as err:
            print("Remote Error: " + str(err))
            conn_str = MONGO_HOST_LOCAL
            mongoDB = MONGO_DB_LOCAL
            try:
                self.client = MongoClient(conn_str)
                self.client.server_info()
                #print("Connection successful to local mongo host")
            except ServerSelectionTimeoutError as errLocal:
                print("Local Error: " + str(errLocal))
        return self.client

    def __exit__(self, exception_type, exception_value, exception_traceback):
        self.client.close()
        del self.client

def unpack_zip(src, dest):
    """takes files in zip folder from src and extracts them to dest"""
    with zipfile.ZipFile(src, 'r') as zip_ref:
        zip_ref.extractall(dest)

def csv_to_mongo(folder, files, map_key):
    """Fetching data from interesting files in csv folder"""
    # kill existing collection if it exists:
    with MongoContext() as client:
        db = client[mongoDB]
        collection = db[UEFA_RAW_DATA]
        collection.drop()

        for idx, file in enumerate(files):
            df = pd.read_csv(f"{folder}/{file}")
            data = df.to_dict(orient='records')
            if idx == 0:
                collection.insert_many(data)
            else:
                for row in data:
                    query = {map_key:  row[map_key]}
                    new_values = {"$set": row}
                    collection.update_one(query, new_values)

def read_from_mongo():
    with MongoContext() as client:
        db = client[mongoDB]
        collection = db[UEFA_RAW_DATA]

        data = collection.find()
        for x in data:
            print("==========================================================================")
            print(x)

def collect_from_kaggle():

    # guard, in case data is already in database.
    with MongoContext() as client:
        db = client[mongoDB]
        if UEFA_RAW_DATA in db.list_collection_names():
            print(f"{UEFA_RAW_DATA} is already in database")
            return False

    unpack_zip(UEFA_ZIP, UEFA_UNZIPPED)
    csv_to_mongo(UEFA_UNZIPPED, UEFA_FILES, "player_name")
    read_from_mongo()

collect_from_kaggle()

Remote Error: pt-n20.p4001.w3.cs.technikum-wien.at:4001: [Errno 54] Connection reset by peer, Timeout: 30s, Topology Description: <TopologyDescription id: 64490b00d96a51d7f5cbf77d, topology_type: Unknown, servers: [<ServerDescription ('pt-n20.p4001.w3.cs.technikum-wien.at', 4001) server_type: Unknown, rtt: None, error=AutoReconnect('pt-n20.p4001.w3.cs.technikum-wien.at:4001: [Errno 54] Connection reset by peer')>]>
{'_id': ObjectId('64490b1ed96a51d7f5cbf780'), 'player_name': 'Courtois', 'club': 'Real Madrid', 'position': 'Goalkeeper', 'minutes_played': 1230, 'match_played': 13, 'goals': 0, 'assists': 0, 'distance_covered': '64.2', 'cross_accuracy': 0, 'cross_attempted': 0, 'cross_complted': 0, 'freekicks_taken': 27, 'pass_accuracy': 76.7, 'pass_attempted': 483, 'pass_completed': 365, 'serial': 447}
{'_id': ObjectId('64490b1ed96a51d7f5cbf781'), 'player_name': 'Vinícius Júnior', 'club': 'Real Madrid', 'position': 'Forward', 'minutes_played': 1199, 'match_played': 13, 'goals': 4, 'assists

## Transfermarkt Spider
<img src="imgs/Transfermarkt_logo.png" style="width:50px; height:auto"/>

In Order to collect insights on players nationality we are querying the website https://transfermarkt.com.
We then parse the html response to collect full_name, nationality, icon and market_value from it

In [100]:
try:
    import parsel
except ImportError:
    !pip install parsel
    import parsel


def transfermarkt_spider(name):
    """queries transfermarkt.com and parses response table"""
    ua = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
    header = {"user-agent": ua}
    result = None
    with requests.Session() as session:
        url = "https://www.transfermarkt.com/schnellsuche/ergebnis/schnellsuche"
        req = session.get(url, params={"query": name}, headers=header)
        response = parsel.Selector(req.text)
        try:
            row = response.xpath("//table[@class='items']/tbody/tr[1]")
            icon_url = row.xpath(".//table//img/@src").get()
            name = row.xpath(".//table//img/@title").get()
            national = row.xpath("./td[5]/img[1]/@alt").get()
            value = row.xpath("./td[6]/text()").get()
            result = dict(icon=icon_url, full_name=name, nationality=national, market_value=value)
        except:
            pass
    return result

# Collect from Transfermarkt.com
with MongoContext() as client:
    db = client[mongoDB]
    collection = db[UEFA_RAW_DATA]
    raw = collection.find()
    count = 0

    mongo_rows = collection.find({ "nationality": { "$exists":False }})
    for mongo_player in mongo_rows:
        print(mongo_player["player_name"])
        if not mongo_player.get("nationality"):
            name = mongo_player["player_name"]
            transfer_data = transfermarkt_spider(name)
            if transfer_data:
                print(transfer_data)
                collection.update_one({"player_name": name}, {"$set": transfer_data})

Mendy
{'icon': 'https://img.a.transfermarkt.technology/portrait/small/157495-1598301159.jpg?lm=1', 'full_name': 'Benjamin Mendy', 'nationality': 'France', 'market_value': '-'}
Martínez
{'icon': 'https://img.a.transfermarkt.technology/portrait/small/44017-1540568493.jpg?lm=1', 'full_name': 'Javi Martínez', 'nationality': 'Spain', 'market_value': '€2.00m'}
Danilo
{'icon': 'https://img.a.transfermarkt.technology/portrait/small/145707-1663577215.jpg?lm=1', 'full_name': 'Danilo', 'nationality': 'Brazil', 'market_value': '€12.00m'}
João Mário
{'icon': 'https://img.a.transfermarkt.technology/portrait/small/149729-1681390479.jpg?lm=1', 'full_name': 'João Mário', 'nationality': 'Portugal', 'market_value': '€14.00m'}
Fernando
{'icon': 'https://img.a.transfermarkt.technology/portrait/small/7767-1447152963.jpg?lm=1', 'full_name': 'Fernando Torres', 'nationality': 'Spain', 'market_value': '-'}
Camara
{'icon': 'https://img.a.transfermarkt.technology/portrait/small/543382-1601103576.png?lm=1', 'full_

### Transform Data
in order to analyse the data we apply type conversions on selected attributes

In [101]:
TYPE_CONVERSIONS = {"minutes_played": "int",
                    'match_played': "int", 'goals': "int", 'assists': "int", 'distance_covered': 'float',
                    'fouls_committed': "int", 'fouls_suffered': "int", 'red': "int", 'yellow': "int",
                    'cross_accuracy': "int", 'cross_attempted': "int", 'cross_complted': "int",
                    'freekicks_taken': "int", 'pass_accuracy': "float", 'pass_attempted': "int",
                    'pass_completed': "int",
                    'balls_recoverd': "int",
                    'clearance_attempted': "int",
                    't_lost': "int",
                    't_won': "int",
                    'tackles': "int"
                    }

def type_converter(item: dict, definitions) -> dict:
    """converts all values that are in a given type key"""
    new_item = dict()
    for k, v in item.items():
        if k in definitions:
            if definitions[k] == "int":
                try:
                    float(v)
                    v = int(v)
                except:
                    v = None
            elif definitions[k] == "float":
                try:
                    v = float(v)
                except:
                    v = None

        new_item[k] = v
    return new_item

def transform_raw_data():
    with MongoContext() as client:
        db = client[mongoDB]
        raw = db[UEFA_RAW_DATA]
        collection = db["players"]
        collection.drop()
        for doc in raw.find():
            cleaned_item = type_converter(doc, TYPE_CONVERSIONS)
            collection.insert_one(cleaned_item)

transform_raw_data()

### Fetch Crime Stats from wikipdedia:
download violent crime stats (intentional homocides) for all countries of the world


In [118]:

def crime_from_wiki():
    """scrape from wikipedia and yield results"""
    url = "https://en.wikipedia.org/wiki/List_of_countries_by_intentional_homicide_rate"
    with requests.Session() as session:
        req = session.get(url)
        response = parsel.Selector(req.text)
        table = response.xpath("//table[contains(@class,'static-row-numbers')]")
        body = table.xpath("./tbody//tr")
        for row in body:
            country = row.xpath("./td[1]//a/text()").get()
            if country:
                country = country.strip("*")
                country = country.strip()
                count_p_100k = float(row.xpath("./td[4]/text()").get())
                yield {"country":country, "count_p_100k":count_p_100k}

# Collect from Wikipedia
with MongoContext() as client:
    db= client[mongoDB]
    collection = db["countries"]
    collection.drop()
    for country in crime_from_wiki():
        collection.insert_one(country)

    for x in collection.find():
        print(x)

{'_id': ObjectId('643c37c7afd7d39c21e6dfc0'), 'country': 'Afghanistan', 'count_p_100k': 6.7}
{'_id': ObjectId('643c37c7afd7d39c21e6dfc1'), 'country': 'Albania', 'count_p_100k': 2.1}
{'_id': ObjectId('643c37c7afd7d39c21e6dfc2'), 'country': 'Algeria', 'count_p_100k': 1.3}
{'_id': ObjectId('643c37c7afd7d39c21e6dfc3'), 'country': 'Andorra', 'count_p_100k': 2.6}
{'_id': ObjectId('643c37c7afd7d39c21e6dfc4'), 'country': 'Angola', 'count_p_100k': 4.8}
{'_id': ObjectId('643c37c7afd7d39c21e6dfc5'), 'country': 'Anguilla', 'count_p_100k': 28.3}
{'_id': ObjectId('643c37c7afd7d39c21e6dfc6'), 'country': 'Antigua and Barbuda', 'count_p_100k': 9.2}
{'_id': ObjectId('643c37c7afd7d39c21e6dfc7'), 'country': 'Argentina', 'count_p_100k': 5.3}
{'_id': ObjectId('643c37c8afd7d39c21e6dfc8'), 'country': 'Armenia', 'count_p_100k': 1.8}
{'_id': ObjectId('643c37c8afd7d39c21e6dfc9'), 'country': 'Aruba', 'count_p_100k': 1.9}
{'_id': ObjectId('643c37c8afd7d39c21e6dfca'), 'country': 'Australia', 'count_p_100k': 0.9}
{'

# Data Collection Pipeline final:
mongo database name: wi21b072

### collections
   #### players:
     all information that from the football players: Connection of UEFA cvs files and Tranfermarkt spider
            {'_id': ObjectId('643c1b35afd7d39c21e6dbf8'), 'player_name': 'Courtois', 'club': 'Real Madrid', 'position': 'Goalkeeper', 'minutes_played': 1230, 'match_played': 13, 'goals': 0, 'assists': 0, 'distance_covered': 64.2, 'cross_accuracy': 0, 'cross_attempted': 0, 'cross_complted': 0, 'freekicks_taken': 27, 'pass_accuracy': 76.7, 'pass_attempted': 483, 'pass_completed': 365, 'serial': 447, 'full_name': 'Thibaut Courtois', 'icon': 'https://img.a.transfermarkt.technology/portrait/small/108390-1665067957.jpg?lm=1', 'market_value': '€60.00m', 'nationality': 'Belgium'}
   #### countries:
        for each country the number of murders per 100k population
        {'_id': ObjectId('643c37c7afd7d39c21e6dfc0'), 'country': 'Afghanistan', 'count_p_100k': 6.7}


